In [1]:
%reload_ext autoreload
%autoreload

## Imports

In [2]:
from function_extractor_v2 import MultiFileFunctionExtractor
from prompt_generator import PromptGenerator
from call_llm import call_llm

## Scrape Data from files

In [3]:
# file_path = "/home/adityadev/GPTDecoder/model.py"  # Replace with the path to your Python file
# file_paths = ["/Users/jaylodha/Desktop/GPTDecoder/model.py", "/Users/jaylodha/Desktop/GPTDecoder/data_loader.py"]  # List of Python files to process
file_paths = ["/Users/jaylodha/Desktop/chillmate/app/backend/chatbot/generate_goals.py"]
output_file = "./function_details_v2.json"  # Output JSON file name

extractor = MultiFileFunctionExtractor(file_paths, output_file)
extractor.extract()

Function details saved to ./function_details_v2.json


## Generate prompt

In [4]:
# Path to the centralized JSON file
json_file = "./function_details_v2.json"

# Example user input
user_prompt = """I need your help in writing a function for a flask based endpoint based on below requirements:

  1. Extract `input_text` from `request.json`, where the input key can be accessed using the key `"input_text"`.
  2. If the extracted input string is empty, return an empty JSON response with a 400 status code.
  3. If the input string is valid, define the following variables:
     - `model_name`: Set to `"llama3-8b-8192"`.
     - `provider_name`: Set to `"groq"`.
  4. Define an extraction JSON object named `ext_json`. Format for the same is present under Relevant Method Details section. Please keep it strictly consistent.
  5. Prepare a context dictionary (`context`).
  6. Use the "generate" method of the "GenerateGoal" class to process the context and generate the required subtasks. Refer to the class and method parameters to integrate it correctly.
  7. Return the result from the "generate" method by converting it into a JSON response with a 200 status code.

  Additional Details:
  - The endpoint should be part of the `"chatbot_bp"` Flask blueprint.
  - Include proper import statements for Flask modules like `request` and `jsonify`, as well as the `GenerateGoal` class.
  - Ensure proper error handling for cases like empty input or unexpected errors during processing.
  - Log the `input_text`, the parsed output, and any errors using `logger.debug` and `logger.error`."""

# Initialize the prompt generator
prompt_generator = PromptGenerator(json_file)

# Generate the final prompt
final_prompt = prompt_generator.generate_prompt(user_prompt)
print(final_prompt)


You are a smart Software Developer who handles large-scale Machine Learning Projects. Your task is to help the user by writing or improving code based on their input. You have access to all relevant class and method details.

The user requested the following:

I need your help in writing a function for a flask based endpoint based on below requirements:

  1. Extract `input_text` from `request.json`, where the input key can be accessed using the key `"input_text"`.
  2. If the extracted input string is empty, return an empty JSON response with a 400 status code.
  3. If the input string is valid, define the following variables:
     - `model_name`: Set to `"llama3-8b-8192"`.
     - `provider_name`: Set to `"groq"`.
  4. Define an extraction JSON object named `ext_json`. Format for the same is present under Relevant Method Details section. Please keep it strictly consistent.
  5. Prepare a context dictionary (`context`).
  6. Use the "generate" method of the "GenerateGoal" class to proc

## LLM Call

In [5]:
model_name = "llama-3.3-70b-versatile"
out, time_taken = call_llm(system_msg=final_prompt, model_name=model_name, seed= 200)

2024-12-08 14:32:24.587 | DEBUG    | call_llm:call_llm:16 - Model being used: llama-3.3-70b-versatile


In [6]:
out

'{   "code": "\\n      from flask import Blueprint, request, jsonify\\n      from your_module import GenerateGoal\\n      import logging\\n\\n      chatbot_bp = Blueprint(\'chatbot_bp\', __name__)\\n      logger = logging.getLogger(__name__)\\n\\n      @chatbot_bp.route(\'/generate_subtasks\', methods=[\'POST\'])\\n      def generate_subtasks():\\n          try:\\n              input_text = request.json.get(\'input_text\')\\n              if not input_text:\\n                  return jsonify({}), 400\\n\\n              model_name = \'llama3-8b-8192\'\\n              provider_name = \'groq\'\\n\\n              ext_json = {\\n                  \'goal\': input_text,\\n                  \'subtasks\': []\\n              }\\n\\n              context = {\\n                  \'user_query\': input_text,\\n                  \'ext_json\': ext_json\\n              }\\n\\n              generate_goal = GenerateGoal(model_name, provider_name)\\n              result = generate_goal.generate(context)\\

In [7]:
import json
out_json = json.loads(out)

In [8]:
out_json.keys()

dict_keys(['code'])

In [9]:
print(out_json['code'])


      from flask import Blueprint, request, jsonify
      from your_module import GenerateGoal
      import logging

      chatbot_bp = Blueprint('chatbot_bp', __name__)
      logger = logging.getLogger(__name__)

      @chatbot_bp.route('/generate_subtasks', methods=['POST'])
      def generate_subtasks():
          try:
              input_text = request.json.get('input_text')
              if not input_text:
                  return jsonify({}), 400

              model_name = 'llama3-8b-8192'
              provider_name = 'groq'

              ext_json = {
                  'goal': input_text,
                  'subtasks': []
              }

              context = {
                  'user_query': input_text,
                  'ext_json': ext_json
              }

              generate_goal = GenerateGoal(model_name, provider_name)
              result = generate_goal.generate(context)

              logger.debug(f'Input Text: {input_text}')
              logger.debug(f'Parsed 